## Import CSVs

In [5]:
import numpy as np 
import pandas as pd 
import sys, requests, shutil, os
data=pd.read_csv('../Data/train.csv')
test_data = pd.read_csv('../Data/test.csv')
data.head(5)

FileNotFoundError: [Errno 2] File b'../Data/train.csv' does not exist: b'../Data/train.csv'

## Sample the data

In [ ]:
landmark_list = [str(x) for x in list(range(1000,3000))]
data_sample = data[data['landmark_id'].isin(landmark_list)]
# data_sample.landmark_id.value_counts()

In [ ]:
# Check data distribution
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pyplot import subplots

colors = np.array(['#4285f4','#34a853','#fbbc05','#ea4335'])
#Define the order in which to display the graph
order = ['1-5','5-10','10-50','50-100','100-200','200-500','>=500']
f, (ax1, ax2) = plt.subplots(1, 2,figsize=(15,5))


def plot_distribution(data_f, data_k, axis):
    # data['landmark_id'].value_counts()
    x=data_f.landmark_id.value_counts().index
    y=pd.DataFrame(data_f.landmark_id.value_counts())

    #Create a variable to group the number of image sin each class
    y['Number of images'] = np.where(y['landmark_id']>=500,'>=500',y['landmark_id'])
    y['Number of images'] = np.where((y['landmark_id']>=200) & (y['landmark_id']<500),'200-500',y['Number of images'])
    y['Number of images'] = np.where((y['landmark_id']>=100) & (y['landmark_id']<200),'100-200',y['Number of images'])
    y['Number of images'] = np.where((y['landmark_id']>=50) & (y['landmark_id']<100),'50-100',y['Number of images'])
    y['Number of images'] = np.where((y['landmark_id']>=10) & (y['landmark_id']<50),'10-50',y['Number of images'])
    y['Number of images'] = np.where((y['landmark_id']>=5) & (y['landmark_id']<10),'5-10',y['Number of images'])
    y['Number of images'] = np.where((y['landmark_id']>=0) & (y['landmark_id']<5),'1-5',y['Number of images'])

    y['Number of images'].value_counts().loc[order].plot(kind = 'bar',color = colors,width = 0.8, ax=axis)
    axis.set_xlabel('Number of images')
    axis.set_ylabel('Number of classes')
    axis.set_title(data_k)
    
plot_distribution(data, 'Original', ax1)
plot_distribution(data_sample, 'Sample', ax2)

In [1]:
import numpy as np 
import pandas as pd 
import sys, requests, shutil, os

In [29]:
fds=pd.read_csv('filtered_dataset.csv')

In [30]:
fds=pd.read_csv('filtered_dataset.csv', header=None)
fds.columns = ['id','url','landmark_id']
fds.head(5)

,id,url,landmark_id
0,0b7ca98b7ff8c0cd,https://lh6.googleusercontent.com/-jCiqpASe2Sk...,25
1,5a2ad55fde17aae8,http://lh3.ggpht.com/-BnZ-muewuJY/Rg_IQKa3G3I/...,25
2,20a181db77ea9702,http://lh4.ggpht.com/-Mhp2gqJwRTQ/SK7Te79-OQI/...,25
3,66db1b12a0e191be,https://lh4.googleusercontent.com/-nnIMghi-_Yc...,25
4,672107f9a8b24aca,http://mw2.google.com/mw-panoramio/photos/medi...,25


In [31]:

data =fds
orig=fds

In [32]:
landmark_list = [str(x) for x in list(range(0,10))]
data_sample = data[data['landmark_id'].isin(landmark_list)]

In [33]:
len(data)

335684

In [34]:
len(data_sample)

36542

## Download Images

In [35]:
#Ref: https://www.kaggle.com/lyakaap/fast-resized-image-download-python-3
import re
TARGET_SIZE = 96 #imports images of resolution 96x96

'''change URLs to resize images to target size'''
def overwrite_urls(df):
    def reso_overwrite(url_tail, reso=TARGET_SIZE):
        pattern = 's[0-9]+'
        search_result = re.match(pattern, url_tail)
        if search_result is None:
            return url_tail
        else:
            return 's{}'.format(reso)
    
    def join_url(parsed_url, s_reso):
        parsed_url[-2] = s_reso
        return '/'.join(parsed_url)
    
    df = df[df.url.apply(lambda x: len(x.split('/'))>1)]
    parsed_url = df.url.apply(lambda x: x.split('/'))
    train_url_tail = parsed_url.apply(lambda x: x[-2])
    resos = train_url_tail.apply(lambda x: reso_overwrite(x, reso=TARGET_SIZE))

    overwritten_df = pd.concat([parsed_url, resos], axis=1)
    overwritten_df.columns = ['url', 's_reso']
    df['url'] = overwritten_df.apply(lambda x: join_url(x['url'], x['s_reso']), axis=1)
    return df
print (len(data_sample))
data_sample_resize = overwrite_urls(data_sample)
print (len(data_sample_resize))

print ('1. URLs overwritten')

'''Split to test and train'''
data_test = pd.DataFrame(columns = ['id','url','landmark_id'])
data_training_all = pd.DataFrame(columns = ['id','url','landmark_id'])
percent_test = 0.01 #takes 1% from each class as holdout data

import random
random.seed(42)
for landmark_id in set(data_sample_resize['landmark_id']):
    n=1
    t = data_sample_resize[(data_sample_resize.landmark_id == landmark_id)] #get all images for a landmark id
    i = 0
    r =[]
    while i < len(t.id):
        it = i
        r.append(t.id.iloc[it])  #create a list of all these images
        i += 1
        
    test = random.sample(r,int(percent_test*len(r))) #randomly pick a sample of 1% images from list 'r'
    training = list(set(r) - set(test))  #get the remaining images
    data_t = data_sample_resize[data_sample_resize.id.isin(test)] #holdout dataset
    data_tr = data_sample_resize[data_sample_resize.id.isin(training)] #training dataset
    data_test = data_test.append(data_t)  
    data_training_all = data_training_all.append(data_tr)
    n+=1

print ('2. train and test set created')


'''Split into train and validation set'''
data_valid = pd.DataFrame(columns = ['id','url','landmark_id'])
data_train = pd.DataFrame(columns = ['id','url','landmark_id'])
percent_validation = 0.2 #takes 20% from each class as holdout data
import random
random.seed(42)
for landmark_id in set(data_training_all['landmark_id']):
    n=1
    t = data_training_all[(data_training_all.landmark_id == landmark_id)]
    i = 0
    r =[]
    while i < len(t.id):
        it = i
        r.append(t.id.iloc[it])
        i += 1
        
    valid = random.sample(r,int(percent_validation*len(r)))
    train = list(set(r) - set(valid)) 
    data_v = data_training_all[data_training_all.id.isin(valid)]
    data_t = data_training_all[data_training_all.id.isin(train)]
    data_valid = data_valid.append(data_v)
    data_train = data_train.append(data_t)
    n+=1

print ('3. train and validation set created')

36542
36542
1. URLs overwritten
2. train and test set created
3. train and validation set created


In [2]:
data_train=pd.read_csv('data_train.csv', header=None)
data_train.columns = ['id','url','landmark_id']
data_valid=pd.read_csv('data_valid.csv', header=None)
data_valid.columns = ['id','url','landmark_id']
data_test=pd.read_csv('data_test.csv', header=None)
data_test.columns = ['id','url','landmark_id']

In [3]:
data_train.head(5)

,id,url,landmark_id
0,1494f337c611e2b1,https://lh3.googleusercontent.com/-tSxUl9SaMTA...,1
1,c709f9eb2e756f2b,http://lh4.ggpht.com/-tVF_zJhzcsw/SMp7PapgLjI/...,1
2,73fd8b6af5e6a56e,http://lh5.ggpht.com/-pe9qqYknGXA/RKdXYTp8ABI/...,1
3,1d7c5801a84eb964,http://lh6.ggpht.com/-FJmIMHfzJr8/S239DBeUwTI/...,1
4,6919c5478cc593d3,https://lh5.googleusercontent.com/-OLAT6SMzwpM...,1


In [4]:
print (len(data_train))
print (len(data_valid))
print (len(data_test))

212741
53160
2657


Create directories 'train_images_model', 'validation_images_model', 'test_images_from_train' inside 'Data' folder before running the code ahead.

In [5]:
# def fetch_image(path,folder):
#     url=path
#     response=requests.get(url, stream=True)
#     with open('Data/' + folder + '/image.jpg', 'wb') as out_file:
#         shutil.copyfileobj(response.raw, out_file)
#     del response
    
# print('ssss')
# '''TRAIN SET - fetch images for the resized URLs and save in the already created directory train_images_model'''
# i=0
# for link in data_train['url']:              #looping over links to get images
#     print(i)
#     if os.path.exists('Data/train_images_model/'+str(data_train['id'].iloc[i])+'.jpg'):
#         i+=1
#         continue
#     fetch_image(link,'train_images_model')
#     os.rename('Data/train_images_model/image.jpg','Data/train_images_model/'+ str(data_train['id'].iloc[i])+ '.jpg')
#     i+=1
#    # print('s',i)
#     #if(i>60):  
#      #   print('sss')
#       #  break
# print('4. train images fetched')

    
# i=0
# for link in data_valid['url']:              #looping over links to get images
#     print(i)
#     if os.path.exists('Data/validation_images_model/'+str(data_valid['id'].iloc[i])+'.jpg'):
#         i+=1
#         continue
#     fetch_image(link,'validation_images_model')
#     os.rename('Data/validation_images_model/image.jpg','Data/validation_images_model/'+ str(data_valid['id'].iloc[i])+ '.jpg')
#     i+=1
#    # if(i>50):   #uncomment to test in your machine
#     #     break
# print('5. Validation images fetched')

# i=0
# for link in data_test['url']:              #looping over links to get images
#     print(i)
#     if os.path.exists('Data/test_images_from_train/'+str(data_test['id'].iloc[i])+'.jpg'):
#         i+=1
#         continue
#     fetch_image(link,'test_images_from_train')
#     os.rename('Data/test_images_from_train/image.jpg','Data/test_images_from_train/'+ str(data_test['id'].iloc[i])+ '.jpg')
#     i+=1
#     #if(i>50):   #uncomment to test in your machine
#     #     break
# print('6. Test images fetched')


+## Data Preprocessing
Creating folders for each landmark ID (Class label)

In [140]:
##create folders for landmark IDs in Training folder
import pandas as pd
import os
import shutil
from shutil import copyfile
import urllib

train_data = data_train

temp = pd.DataFrame(data_train.landmark_id.value_counts())
temp.reset_index(inplace=True)
temp.columns = ['landmark_id','count']

def createfolders(dataset,folder):
    i = 0
    while i < len(dataset):
        landmark = str(dataset.landmark_id.iloc[i])
        path = 'Data/' + folder + '/'+ landmark
        if not os.path.exists(path):
            os.makedirs(path)
        i+=1
createfolders(temp,'train_images_model')

print ('Train folders created')

rootdirpics = r'Data/train_images_model/'
rootdirfolders = r'Data/train_images_model/'

def transformdata(data,path1, path2):
#     print (data)
    n = 1
    for landmark_id in set(data['landmark_id']):
        t = data[(data.landmark_id == landmark_id)]
        i = 0
        r =[]
        while i < len(t.id):
            it = i 
            r.append(t.id.iloc[it])
            i += 1
        if len(t.id)!=len(r):
            print(len(t.id),len(r))
        for files in os.listdir(rootdirpics):    # loop through startfolders
            inpath = path1 + files
            folder = str(int(landmark_id))
            outpath = path2 + folder  
            if ((files.split('.')[0] in r) & (os.path.getsize(inpath) >1000)):
#                 print('move')
                shutil.move(inpath, outpath)
            elif ((files.split('.')[0] in r) & (os.path.getsize(inpath) <= 1000)):
                os.remove(inpath)
#             else:
#                 print(files.split('.'))
        n+=1

transformdata(data_train,rootdirpics, rootdirfolders)
print ('Train images moved')

Train folders created
Train images moved


In [138]:
##create folders for landmark IDs in Validation folder

temp = pd.DataFrame(data_valid.landmark_id.value_counts())
temp.reset_index(inplace=True)
temp.columns = ['landmark_id','count']
createfolders(temp,'validation_images_model')
print(len(temp))
print ('Validation folders created')

# make folders for landmark ID which had no images in validation sets - required for codes running next
# available = [int((x[0].split('/'))[-1]) for x in os.walk(r'Data/validation_images_model/') if len((x[0].split('/'))[-1]) > 0]
# new = [str(x) for x in range(1,51) if x not in available]
# for i in new:
#     path = 'Data/validation_images_model/' + i
#     if not os.path.exists(path):
#         os.makedirs(path)

rootdirpics = r'Data/validation_images_model/'
rootdirfolders = r'Data/validation_images_model/'
transformdata(data_valid,rootdirpics, rootdirfolders)
print ('Validation images moved')

50
Validation folders created
Validation images moved


## Display Images

In [141]:
from IPython.display import Image
from IPython.core.display import HTML 

def displayLandmarkImagesLarge(urls, category_name):
    img_style = "width: 200px; height:160px; margin: 0px; float: left; border: 1px solid black;"
    images_list = ''.join([f"<img style='{img_style}' src='{u}' />" for _, u in urls.head(12).iteritems()])
    display(HTML(images_list))

category = data['landmark_id'].value_counts().keys()[15]
urls = data[data['landmark_id'] == category]['url']
displayLandmarkImagesLarge(urls, "")

In [142]:
pd.Series(data['landmark_id'].value_counts().keys())[1:4]

1    6051
2    6599
3      15
dtype: int64

### Predictions using weights from ImageNet

Before proceeding with any model training, VGG16 architecture trained on Imagenet dataset is picked up and is used to predict on the images that we had. Observation: Imagenet weights are successfully able to capture the generic features from landmarks.

In [43]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

model =VGG16(weights = 'imagenet', include_top = True)

img_path = r'Data\train_images_model\2338\0fecac19aec6d24c.jpg'
img = image.load_img(img_path, target_size= (224,224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)
x = preprocess_input(x)

preds = model.predict(x)
print ('Predicted:', decode_predictions(preds, top = 3)[0])

# 0a0667fc2436a04b = stupa (17%)
# 0b48dd3b81d5b6ba = triumphal_arch(25%)

FileNotFoundError: [Errno 2] No such file or directory: 'Data\\train_images_model\\2338\\0fecac19aec6d24c.jpg'

In [22]:
# available = [int((x[0].split('/'))[-1]) for x in os.walk(r'Data/train_images_model/') if len((x[0].split('/'))[-1]) > 0]
# new = [str(x) for x in range(1,14000) if x not in available]
# for i in new:
#     path = 'Data/train_images_model/' + i
#     if not os.path.exists(path):
#         os.makedirs(path)

# Model Training - Transfer Learning

In [6]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import Model
import csv
import os
import cv2
from keras.models import load_model
import matplotlib.pyplot as plt
import math
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import tensorflow as tf
import keras

Using TensorFlow backend.


In [14]:
train_data_dir = 'Data/train_images_model'
validation_data_dir = 'Data/validation_images_model'

#counts the number of files in subdirectories of a given directory. 
#outputs the count of images and label of each image in order of reading.

def count(dir):
    i = 1
    count = []
    while i <= 50:
    
        f = str(i)
        #print (f)
        for root, dirs, files in os.walk(dir +'/'+ f):  # loop through startfolders
            for pic in files:
                count.append(f)   
        i += 1
    print (len(count))
    return ([len(count),count])

nb_train_samples = count(train_data_dir)
nb_validation_samples = count(validation_data_dir)
print(nb_train_samples)
print(nb_validation_samples)

# nb_train/validation_samples is a list of lists of the format [len(count),count]. len(count) is the total number of images in 
# train/validation folder. count is a list of class labels i.e. landmark IDs of the images read.

200000
50000
[200000, ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1

In the next steps, a batch size will be defined, which needs to be a factor of both the number of train and validation images. I had to remove a certain number of images in the process. Use the code below to move some images out of the train/validation folder in order to get a common factor in the range of 100-1000 (approx.). This common factor will be used as the batch size going forward.
A folder named 'Data/moved_images' has to be created before executing the below. 

In [15]:
# CL = '2338' #class from which to move images
# NI = 50 #Number of images to be moved

# i = 0
# for files in os.listdir(r'Data/train_images_model/' + CL):    # loop through startfolders
#             i+=1
#             inpath = r'Data/train_images_model/' + CL + '/' + files
#             outpath = r'Data/moved_images' 
#             shutil.move(inpath, outpath)
#             if i == NI:
#                 break

#### Converting images to vectors using weights from ImageNet on VGG16

In [ ]:
## Had to remove images (1046 - 6c39ce7ee33cea49.jpg)(1310 - f2bf3d0a94733000.jpg) (1834 - 35494ec38ca8b6ff.jpg) 
##(1946 - aa108073efc35e13.jpg) (2729 - 43608f66f2f680e3.jpg) (2975 - bf4b45efc3dfda07.jpg) 
# because were throwing error while running the code.

img_width, img_height = 96, 96 # dimensions of downloaded images.
top_model_weights_path = 'bottleneck_fc_model.h5' # A file with this name would be saved later in the code
epochs = 5
batch_size = 1000 #As found in the previous step
def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255,
                                 rotation_range=30,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 zoom_range = 0.5,
                                 brightness_range = [0.5,1.5])

#     build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet', input_shape=(96,96,3))
    print(model.summary())
    print ('start1')
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None, # this means our generator will only yield batches of data, no labels
        shuffle=False) # our data will be in order
    
#     the predict_generator method returns the output of a model, given a generator that yields batches of numpy data
    print ('start2')
    bottleneck_features_train = model.predict_generator(generator, nb_train_samples[0] // batch_size) #nb_train_samples[0]
    print ('bottleneck_features_trained')
    print(nb_train_samples[0])
    print (bottleneck_features_train)
    
    
    with open('bottleneck_features_train.npy', 'wb') as features_train_file:
        np.save(features_train_file, bottleneck_features_train)
    print ('Train done')
    

## Had to remove (2061 - a32ee99990779b22.jpg) because was throwing an error
    datagen = ImageDataGenerator(rescale=1. / 255) #No image augmentation in validation dataset
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    print ('validation predict start')
    bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples[0] // batch_size)
    print (bottleneck_features_validation)

    with open('bottleneck_features_validation.npy', 'wb') as features_validation_file:
        np.save(features_validation_file, bottleneck_features_validation)
    
    print ('validation done')
save_bottleneck_features()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0     

#### Initializing the weights on top 3 layers

In [26]:
epochs = 5
batch_size = 1000
import numpy as np

def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    train_labels = np.array(nb_train_samples[1])
    train_labels = [str(int(train_label) - 1) for train_label in train_labels] 
    #Had to subtract 1000 because class labels should start from 0. In this case, class labels had a range from 1000 to 2999. 
#     print (train_labels)
    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
    validation_labels = np.array(nb_validation_samples[1])
    validation_labels = [str(int(validation_label) - 1) for validation_label in validation_labels]
    
#     print (validation_labels)
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    n_class = 50 #number of classes fed to the model
    model.add(Dense(n_class, activation='softmax'))
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy', metrics=['accuracy'])
    train_labels = to_categorical(train_labels, n_class)
    validation_labels = to_categorical(validation_labels, n_class)
    
    
    print ('model fit starting')
    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

train_top_model()

model fit starting
Train on 200000 samples, validate on 50000 samples
Epoch 1/5
200000/200000 [==============================] - 18s 92us/step - loss: 2.9236 - acc: 0.2314 - val_loss: 2.3975 - val_acc: 0.3420
Epoch 2/5
200000/200000 [==============================] - 18s 88us/step - loss: 2.5160 - acc: 0.3095 - val_loss: 2.3102 - val_acc: 0.3594
Epoch 3/5
200000/200000 [==============================] - 18s 88us/step - loss: 2.3543 - acc: 0.3437 - val_loss: 2.1185 - val_acc: 0.3955
Epoch 4/5
200000/200000 [==============================] - 18s 88us/step - loss: 2.2487 - acc: 0.3653 - val_loss: 2.0768 - val_acc: 0.3979
Epoch 5/5
200000/200000 [==============================] - 18s 88us/step - loss: 2.1699 - acc: 0.3837 - val_loss: 1.9690 - val_acc: 0.4295


#### Compile and train the entire model

In [27]:
 img_width, img_height = 96, 96
top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'Data/train_images_model'
validation_data_dir = 'Data/validation_images_model'
batch_size = 100
epochs = 15
def trainCNN():

    # build the VGG16 network

    base_model = applications.VGG16(weights='imagenet',include_top= False,input_shape=(96,96,3))

    top_model = Sequential()
    top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dense(256, activation='relu'))
    n_class = 50
    top_model.add(Dense(n_class, activation='softmax'))
    top_model.load_weights(top_model_weights_path) #Load the weights initialized in previous steps
    

    model = Model(input= base_model.input, output= top_model(base_model.output))
    
    # set the first 16 layers to non-trainable (weights will not be updated) - 1 conv layer and three dense layers will be trained
    for layer in model.layers[:16]:
        layer.trainable = False

    # compile the model with a SGD/momentum optimizer and a very slow learning rate.
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=0.0001, beta_1=0.9,beta_2=0.999,epsilon=1e-8, decay=0.0),
                  metrics=['accuracy'])
    print ('Compilation done.')
    
    train_datagen = ImageDataGenerator(rescale=1. / 255,
                                       rotation_range=90,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        zoom_range = 0.5)
    
    valid_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

    np.save('class_indices.npy', train_generator.class_indices)

    validation_generator = valid_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')
    
    print ('Model fit begins...')
    model.fit_generator(
        train_generator,
        steps_per_epoch=340,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=150,
        callbacks=[ModelCheckpoint(filepath=top_model_weights_path, save_best_only=True, save_weights_only=True)]
        )
    
    model.save_weights(top_model_weights_path)

trainCNN()

C:\Users\User\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`


Compilation done.
Found 200000 images belonging to 50 classes.
Found 50000 images belonging to 50 classes.
Model fit begins...
Epoch 1/15
340/340 [==============================] - 3085s 9s/step - loss: 2.8669 - acc: 0.3004 - val_loss: 2.1786 - val_acc: 0.4733
Epoch 2/15
340/340 [==============================] - 2947s 9s/step - loss: 2.2133 - acc: 0.4422 - val_loss: 1.8049 - val_acc: 0.5365
Epoch 3/15
340/340 [==============================] - 2941s 9s/step - loss: 1.9281 - acc: 0.4998 - val_loss: 1.4742 - val_acc: 0.6089
Epoch 4/15
340/340 [==============================] - 2968s 9s/step - loss: 1.7630 - acc: 0.5388 - val_loss: 1.3323 - val_acc: 0.6431
Epoch 5/15
340/340 [==============================] - 2980s 9s/step - loss: 1.6477 - acc: 0.5663 - val_loss: 1.2727 - val_acc: 0.6631
Epoch 6/15
340/340 [==============================] - 3135s 9s/step - loss: 1.5480 - acc: 0.5876 - val_loss: 1.1455 - val_acc: 0.6898
Epoch 7/15
340/340 [==============================] - 3014s 9s/step -

# Predictions

In [28]:
from tensorflow.python.platform import app
import argparse
import os
import sys
import time
from time import *
import io
import tensorflow as tf

In [29]:
top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'Data/train_images_model'
testfile = 'Data/test_images_from_train'

subfile = 'Data/sub_spezifinal.csv'

def count(dir):
    i = 1
    count = []
    while i <= 9:
        f = str(i)
        for root, dirs, files in os.walk(dir +'/'+ f):  # loop through startfolders
            for pic in files:
                count.append(pic)

        i += 1
    return len(count)

nb_train_samples = count(train_data_dir)
nb_validation_samples = count(validation_data_dir)

In [40]:
from keras import backend as K 
from keras.applications.vgg16 import preprocess_input, decode_predictions
def predict(image_path):
    print ('starting...')
    path, dirs, files = next(os.walk(image_path))
    file_len = len(files)
    print('Number of Testimages:', file_len)

    train_datagen = ImageDataGenerator(rescale=1. / 255)

    generator = train_datagen.flow_from_directory(train_data_dir, batch_size=batch_size)
    label_map = (generator.class_indices)

    n_class = 50+

    with open(subfile, 'w') as csvfile:
        newFileWriter = csv.writer(csvfile)
        newFileWriter.writerow(['id', 'landmarks'])

        file_counter = 0
        for root, dirs, files in os.walk(image_path):  # loop through startfolders
            for pic in files:
                t1 = clock()

                #loop folder and convert image
                path = image_path + '/' + pic


                orig = cv2.imread(path)
                image = load_img(path, target_size=(96, 96))
                image = img_to_array(image)

                # important! otherwise the predictions will be '0'
                image = image / 255

                image = np.expand_dims(image, axis=0)

                #classify landmark
                base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(96, 96, 3))

                top_model = Sequential()
                top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
                top_model.add(Dense(256, activation='relu'))
                top_model.add(Dense(256, activation='relu'))
                top_model.add(Dense(n_class, activation='softmax'))

                model = Model(input=base_model.input, output=top_model(base_model.output))
                model.load_weights(top_model_weights_path)

                prediction = model.predict(image)
                #print(prediction)
                
                #print ('Predicted:', decode_predictions(prediction, top = 3)[0])
                
                class_predicted = prediction.argmax(axis=1)
                #class_predicted = np.argmax(prediction,axis=1)
                #print (pic, class_predicted)


                inID = class_predicted[0]
                #print (inID)

                inv_map = {v: k for k, v in label_map.items()}
#                 print (class_dictionary)

                label = inv_map[inID]
                #print(label)

                score = max(prediction[0])
                scor = "{:.2f}".format(score)
                out = str(label) + ' '+ scor

                #print (score)

                newFileWriter.writerow([os.path.splitext(pic)[0], out])
                print (os.path.splitext(pic)[0], scor,data_test[(data_test.id == os.path.splitext(pic)[0])].landmark_id,label)

                K.clear_session()

testfile = 'Data/test_images_from_train'
predict(testfile)



starting...
Number of Testimages: 2509
Found 200000 images belonging to 50 classes.


C:\Users\User\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\User\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:47: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`


0018bf21640251c6 1.00 1595    20
Name: landmark_id, dtype: int64 20
001c033903569272 0.97 1981    33
Name: landmark_id, dtype: int64 33
001fa4413e920054 0.35 394    6
Name: landmark_id, dtype: int64 2
003adb018cacfc24 0.67 1085    22
Name: landmark_id, dtype: int64 22
0045e233b30d643d 1.00 736    35
Name: landmark_id, dtype: int64 35
00477c4ff62aadf9 0.73 1912    46
Name: landmark_id, dtype: int64 6
007c14cd9f08288e 1.00 798    35
Name: landmark_id, dtype: int64 35
007fa487ef5f75f0 0.86 345    6
Name: landmark_id, dtype: int64 6
00b5c599f7df5ba4 0.94 2131    24
Name: landmark_id, dtype: int64 24
00d2d939797ac18c 0.97 799    35
Name: landmark_id, dtype: int64 35
00dca7c1b86092c0 0.77 2363    34
Name: landmark_id, dtype: int64 34
00e1f16ce16a20d1 0.50 972    22
Name: landmark_id, dtype: int64 22
00e428c0f2fcd9b8 0.89 2236    15
Name: landmark_id, dtype: int64 15
00e45d9915955021 0.88 23    19
Name: landmark_id, dtype: int64 19
00f1d81104bbc0df 0.98 1978    33
Name: landmark_id, dtype: in

09e3000690f4154e 0.99 0    19
Name: landmark_id, dtype: int64 19
09f1325e1e310d3b 1.00 2431    32
Name: landmark_id, dtype: int64 32
09f6875f13b93fd9 0.62 2561    25
Name: landmark_id, dtype: int64 25
09fd0df7067a4eeb 0.84 1291    22
Name: landmark_id, dtype: int64 22
09fe3bb15e0f5058 0.95 1124    22
Name: landmark_id, dtype: int64 22
0a048e7ea22e82ad 0.96 2121    24
Name: landmark_id, dtype: int64 24
0a055adc63e26a13 1.00 1034    22
Name: landmark_id, dtype: int64 22
0a0d21aed81cfcaf 0.24 2284    49
Name: landmark_id, dtype: int64 49
0a5c3915915c7ce9 0.84 1329    22
Name: landmark_id, dtype: int64 22
0a92a8fcf61d913f 0.98 57    19
Name: landmark_id, dtype: int64 19
0ab340a1feee1485 0.47 2640    30
Name: landmark_id, dtype: int64 17
0ace06597db37cea 1.00 386    6
Name: landmark_id, dtype: int64 6
0afefe93120fa767 0.28 1510    38
Name: landmark_id, dtype: int64 24
0b12cc641a9473d2 1.00 1229    22
Name: landmark_id, dtype: int64 22
0b17d2bf2dd7f34e 0.86 894    35
Name: landmark_id, dtype

14acf8adb8785a95 0.91 833    35
Name: landmark_id, dtype: int64 35
150674ab21228f6d 1.00 1717    36
Name: landmark_id, dtype: int64 36
15193a9895c90e64 0.93 2078    24
Name: landmark_id, dtype: int64 24
1521fbe73a89e755 1.00 1598    20
Name: landmark_id, dtype: int64 20
155b725a00dcca89 0.25 436    2
Name: landmark_id, dtype: int64 17
155c5c45ba6a1f9b 1.00 462    2
Name: landmark_id, dtype: int64 2
15698824bc906133 1.00 1049    22
Name: landmark_id, dtype: int64 22
1573f24c71b2a86e 1.00 913    35
Name: landmark_id, dtype: int64 35
1577892721a9f264 0.30 295    45
Name: landmark_id, dtype: int64 16
158756e026d26dcb 0.22 254    44
Name: landmark_id, dtype: int64 44
159491b13d594f4e 0.93 2013    24
Name: landmark_id, dtype: int64 24
15b275ba9f98735f 1.00 1116    22
Name: landmark_id, dtype: int64 22
15b355c8601f4d3c 1.00 1710    36
Name: landmark_id, dtype: int64 36
15c321a7a79a63aa 1.00 1526    20
Name: landmark_id, dtype: int64 20
15dddee1d7abf511 0.82 2629    30
Name: landmark_id, dtype

...

32fef0b952044f7e 1.00 805    35
Name: landmark_id, dtype: int64 35
330214807bd80884 0.92 2028    24
Name: landmark_id, dtype: int64 24
3304d4b85da0d514 0.67 1274    22
Name: landmark_id, dtype: int64 22
3310f7878a60d327 0.59 2041    24
Name: landmark_id, dtype: int64 24
332918007382bb7a 1.00 1843    9
Name: landmark_id, dtype: int64 9
332ef70db93cef62 1.00 786    35
Name: landmark_id, dtype: int64 35
3344b55856dbb906 1.00 1527    20
Name: landmark_id, dtype: int64 20
334c7aa9cf3a2ba2 1.00 576    35
Name: landmark_id, dtype: int64 35
3377185ca81a8e2b 1.00 1535    20
Name: landmark_id, dtype: int64 20
338d0d8181208502 0.96 1856    9
Name: landmark_id, dtype: int64 9
33a4845bc601013d 0.26 287    45
Name: landmark_id, dtype: int64 50
33c40fcb03902f01 0.25 2001    24
Name: landmark_id, dtype: int64 13
33d0349a78b79e26 1.00 7    19
Name: landmark_id, dtype: int64 19
33dd90ff001d3efd 0.98 1318    22
Name: landmark_id, dtype: int64 22
33e11372c604f522 0.44 1469    1
Name: landmark_id, dtype: i

3f05bd691b4a7e69 0.91 1303    22
Name: landmark_id, dtype: int64 22
3f4d89eff74a8732 0.95 1980    33
Name: landmark_id, dtype: int64 33
3f560b9959ec9f43 1.00 2556    25
Name: landmark_id, dtype: int64 25
3f681a6f00aa72aa 0.51 1442    37
Name: landmark_id, dtype: int64 16
3f6a7d7c3f8ca559 1.00 2037    24
Name: landmark_id, dtype: int64 24
3f91f802e1bcaf80 0.22 54    19
Name: landmark_id, dtype: int64 19
3fa6a44ce82e5199 1.00 1846    9
Name: landmark_id, dtype: int64 9
3fadb1390510a936 1.00 448    2
Name: landmark_id, dtype: int64 2
3fc579c08cff4e06 1.00 1283    22
Name: landmark_id, dtype: int64 22
3fd3bd66f811ee8b 0.98 964    22
Name: landmark_id, dtype: int64 22
3fd74daeaaa8919b 1.00 1618    36
Name: landmark_id, dtype: int64 36
3fde4e3cf3995e31 0.96 2330    4
Name: landmark_id, dtype: int64 4
3ff0fd695f7926be 0.99 749    35
Name: landmark_id, dtype: int64 35
40034bd2146a934e 0.99 1862    9
Name: landmark_id, dtype: int64 9
4006333a0a3699da 0.91 1035    22
Name: landmark_id, dtype: in

49f74c9e5fe9d464 1.00 829    35
Name: landmark_id, dtype: int64 35
49faf0954088baa1 0.98 463    2
Name: landmark_id, dtype: int64 2
4a03625a89133a9e 0.83 1101    22
Name: landmark_id, dtype: int64 22
4a46448d914f69a9 0.58 353    6
Name: landmark_id, dtype: int64 6
4a5b9361532455d0 0.98 1495    47
Name: landmark_id, dtype: int64 47
4a7e1772c223ad4a 1.00 590    35
Name: landmark_id, dtype: int64 35
4a9ee93bd6164a0d 0.49 1630    36
Name: landmark_id, dtype: int64 36
4aa7a92f27190e7e 1.00 571    35
Name: landmark_id, dtype: int64 35
4ab89eb72ff5652f 1.00 1747    36
Name: landmark_id, dtype: int64 36
4ac32b8edaa54ad8 0.30 2545    25
Name: landmark_id, dtype: int64 25
4acc6d3721af0568 1.00 672    35
Name: landmark_id, dtype: int64 35
4add4bcc8dc8974b 0.81 814    35
Name: landmark_id, dtype: int64 35
4aebd651b758a67d 0.29 2148    24
Name: landmark_id, dtype: int64 26
4af90fa7d02ce425 0.84 1414    26
Name: landmark_id, dtype: int64 26
4b079962bdb09c38 1.00 1011    22
Name: landmark_id, dtype: 

54156ce0bb39798f 0.79 2085    24
Name: landmark_id, dtype: int64 24
544f38202a9899b0 0.98 2505    18
Name: landmark_id, dtype: int64 18
546057865acc2f44 1.00 1297    22
Name: landmark_id, dtype: int64 22
546fd79d59201b54 1.00 2606    30
Name: landmark_id, dtype: int64 30
5497a0ab06bae228 0.34 1425    13
Name: landmark_id, dtype: int64 17
54d3341802a8c818 0.80 2272    49
Name: landmark_id, dtype: int64 49
551558e8f8c96dac 1.00 2515    18
Name: landmark_id, dtype: int64 18
5546ee565e707907 0.56 2222    15
Name: landmark_id, dtype: int64 41
55649a3b871fbddb 1.00 1198    22
Name: landmark_id, dtype: int64 22
55847e71f16f2203 0.86 1707    36
Name: landmark_id, dtype: int64 36
55a6d744d1d6ec9e 0.76 304    6
Name: landmark_id, dtype: int64 6
55b6714dde2c87e5 0.78 1557    20
Name: landmark_id, dtype: int64 13
55ba74ddbaad08e4 0.95 1822    9
Name: landmark_id, dtype: int64 9
56317db39959c3a3 0.97 1323    22
Name: landmark_id, dtype: int64 22
5634b3f6d38db5cd 0.14 966    22
Name: landmark_id, dt

KeyboardInterrupt: 

In [52]:
 toco \ --output_file=trained_model.tflite \ --keras_model_file=bottleneck_fc_model.h5

SyntaxError: unexpected character after line continuation character (<ipython-input-52-6c9e540e9845>, line 1)

In [43]:
pip install toco


Note: you may need to restart the kernel to use updated packages.
